In [1]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from Bio import SeqIO

In [12]:
df = pd.read_csv("raw_data/metadata_600.csv")
for i in range(600):
    #print(protien_names_ordered_filtered[i][3])
    pass

print(df.loc[df['RMS_Name'] == 'Aap10433IP']['Methylation Motif'])

0    CGANNNNNNNRTAG/CTAYNNNNNNNTCG
Name: Methylation Motif, dtype: object


In [26]:
df.loc[df['RMS_Name'] == 'Aap5906IP']['Methylation Motif'].tolist()[0]
# find the location where RMS_Name is Aap10433IP. 
# Then get what is in Methylation Motif colum. 
# This is a list of length 1. So just use [0] to get the value we want.

# make list of strings that is sorted accordingly to protien_names_ordered_filtered[i]


'CCANNNNNNRTAC/GTAYNNNNNNTGG'

In [29]:
MethylationMotif = []
for i in range(600):
    MethylationMotif.append( df.loc[df['RMS_Name'] == protien_names_ordered_filtered[i][3]]['Methylation Motif'].tolist()[0] )
print(len(MethylationMotif))

600


In [47]:
bas4_dict = {
    'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T': [0,0,0,1],
    'R': [1,0,1,0], 'Y': [0,1,0,1], 'K': [0,0,1,1], 'M': [1,1,0,0], 'S': [0,1,1,0], 'W': [1,0,0,1],
    'B': [0,1,1,1], 'D': [1,0,1,1], 'H': [1,1,0,1], 'V': [1,1,1,0], 'N': [1,1,1,1] 
}
def str_to_base4code(string):
    coded = []
    for char in string:
        coded = coded + bas4_dict[char]
    return coded
# functions for building list of encoded strings and padding the lists
def one_hot(data_1d):
    data_1d_encoded = []
    for string in data_1d:
        data_1d_encoded.append(str_to_base4code(string))
    return data_1d_encoded
def pad(data_1d):
    max_length = len(max(data_1d, key=len))
    padded_data = []
    for i, line in enumerate(data_1d):
        padded_data.append(line + [0]*(max_length - len(line)))
    return padded_data
df = pd.read_csv("raw_data/metadata_600.csv")

# we will split the data by "left / right" Data: CGANNNNNNNRTAG/CTAYNNNNNNNTCG
MethylationMotif = []
for i in range(600):
    MethylationMotif.append( df.loc[df['RMS_Name'] == protien_names_ordered_filtered[i][3]]['Methylation Motif'].tolist()[0] )
left_list = []
right_list = []
for line in MethylationMotif:
    try: left, right = line.rsplit('/')
    except: left, right = line.rsplit('/')[0], '' # if there is no "/" then we will just fill "right" with ''
    left_list.append(left)
    right_list.append(right)

left_list_onehot_padded = pad(one_hot(left_list))
right_list_onehot_padded = pad(one_hot(right_list))

list_combined = []
for l,f in zip(left_list_onehot_padded, right_list_onehot_padded): list_combined.append(l+f)

torch.save(torch.FloatTensor(list_combined), "data/metalation_motifs.pt")

In [48]:
temp = torch.load("data/metalation_motifs.pt")

In [49]:
temp.size()

torch.Size([600, 136])

In [2]:

motifs_pd = pd.read_csv("raw_data/metadata_600.csv")

# set up directory of how each symbol connects to ACGT: http://www.hgmd.cf.ac.uk/docs/nuc_lett.html
# mapping 'N': [1,1,1,1] because it should accept any base target. Could be [0,0,0,0] because this is just padding...
bas4_dict = {
    'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T': [0,0,0,1],
    'R': [1,0,1,0], 'Y': [0,1,0,1], 'K': [0,0,1,1], 'M': [1,1,0,0], 'S': [0,1,1,0], 'W': [1,0,0,1],
    'B': [0,1,1,1], 'D': [1,0,1,1], 'H': [1,1,0,1], 'V': [1,1,1,0], 'N': [1,1,1,1] 
}

def str_to_base4code(string):
    coded = []
    for char in string:
        coded = coded + bas4_dict[char]
    return coded

# functions for building list of encoded strings and padding the lists
def one_hot(data_1d):
    data_1d_encoded = []
    for string in data_1d:
        data_1d_encoded.append(str_to_base4code(string))
    return data_1d_encoded

def pad(data_1d):
    max_length = len(max(data_1d, key=len))
    padded_data = []
    for i, line in enumerate(data_1d):
        padded_data.append(line + [0]*(max_length - len(line)))
    return padded_data


In [3]:

###### transitioning from pandas dataframe to list of the feature we want
# 600: Do they want to keep their own testing data away from us for when we give them the model?
motif11 = []#first half, left of /
motif12 = []#first half, right of /
for line in motifs_pd['Motif_1stHalf'].to_list():
    motif11.append(line.rsplit('/')[0])
    try:
        motif12.append(line.rsplit('/')[1])
    except:
        motif12.append('')
        print(line, " has no slash 1")
motif21 = []
motif22 = []

for line in motifs_pd['Motif_2ndHalf'].to_list():
    motif21.append(line.rsplit('/')[0])
    try: # sometimes there is no / present
        motif22.append(line.rsplit('/')[1])
    except:
        motif22.append('')
        print(line, " has no slash 2")
###### encoding and then padding each list sequences
motif11 = one_hot(motif11)
motif11 = pad(motif11)
motif12 = one_hot(motif12)
motif12 = pad(motif12)
motif21 = one_hot(motif21)
motif21 = pad(motif21)
motif22 = one_hot(motif22)
motif22 = pad(motif22)
###### one hot encode the number of Ns for the spacing
# compare 'N's on left / right half. double check they are even
same_on_both_sides = True
for line in motifs_pd['Methylation Motif']:
    try:
        a,b = line.split("/")
        if int(a.count('N')) != int(b.count('N')):
            same_on_both_sides = False
    except:
        pass
print("Ns are same on both sides? ", same_on_both_sides)

# Get number of Ns into a one hot
num_ns = []
for line in motifs_pd['Methylation Motif']:
    num_ns.append(int(line.count('N')/2))

num_ns_onehot = []
for n in num_ns:
    num_ns_onehot.append([int(i) for i in np.eye(9)[n-2].tolist()])
print(len(num_ns))
######
# bring together one hot version of motif 1st half part 1, motif 1st half part 2, motif 2nd half part 1, motif 2nd half part 2, and number of spaces (n) 
motifs_onehot = []
for a, b, c, d, n in zip(motif11, motif12, motif21, motif22, num_ns_onehot):
    motifs_onehot.append(a + b + c + d + n)

torch.save(torch.FloatTensor(motifs_onehot), "data/motifs-base4-numN.pt")


GATC  has no slash 1
TCGA  has no slash 1
TCGA  has no slash 2
TTAA  has no slash 2
GATC  has no slash 2
RTAY  has no slash 2
RTAY  has no slash 2
RTAY  has no slash 2
TTAA  has no slash 2
TCGA  has no slash 2
Ns are same on both sides?  True
600


In [1]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [2]:
from Bio import SeqIO

In [3]:
protien_names = {'m':[],'r':[],'s':[],'target':[]}
for seq_record in SeqIO.parse("raw_data/Type_I_M_subunit_genes_Protein_g_clean_sorted.faa", "fasta"):
    protien_names['m'].append(seq_record.id)
for seq_record in SeqIO.parse("raw_data/Type_I_R_subunit_genes_Protein_g_clean_sorted.faa", "fasta"):
    protien_names['r'].append(seq_record.id)
for seq_record in SeqIO.parse("raw_data/Type_I_S_subunit_genes_Protein_g_clean_sorted.faa", "fasta"):
    protien_names['s'].append(seq_record.id)
t = pd.read_csv("raw_data/metadata_600.csv")
for _, row in t.iterrows():
    protien_names['target'].append(row['RMS_Name'])

In [4]:
# check if the order / names all agree
i = 0
for m,r,s,t in zip(protien_names['m'],protien_names['r'],protien_names['s'],protien_names['target']):
    mmod = m[2:]
    if r[-1] == 'P':
        rmod = r[:-1]
    else:
        rmod = r
    if s.startswith('S1'):
        smod = s[3:]
    else:
        smod = s[2:]
    tmod = t[:-1]
    if mmod != rmod or rmod != smod or smod != tmod: 
        print(mmod,rmod,smod,tmod)
        print(m,r,s,t)
        print()
    i += 1
print(i)
'''
Clearly the issue is:
1. names have things pinned onto them. This has been managed.
2. the target data is out of order. 
- Instead of zipping them, pick a protien name and try to find it in m,r,s,t
- If found then add each name variation into new dictionary. 
protien_names_use = {1:["M.Aba78I","S.Aba78I","Aba78IP","Aba78IP"], 2:[...]}
- Now I have a can loop through the dictionary indexs 1,2,3... to pull out the names in each respective files/csv 
and pull in the data to do stuff with. This keeps it ordered and it only takes protiens present in the target.
'''

Aba78I Aba78I Aba78I Aba7804I
M.Aba78I Aba78IP S.Aba78I Aba7804IP

Aba7804I Aba7804I Aba7804I Aba78I
M.Aba7804I Aba7804IP S.Aba7804I Aba78IP

Awo1030II Awo1030II Awo1030II Awo1030III
M.Awo1030II Awo1030IIP S.Awo1030II Awo1030IIIP

Awo1030III Awo1030III Awo1030III Awo1030II
M.Awo1030III Awo1030IIIP S.Awo1030III Awo1030IIP

BceSVI BceSVI BceSVI BceS28I
M.BceSVI BceSVIP S.BceSVI BceS28IP

BceS28I BceS28I BceS28I BceSVI
M.BceS28I BceS28IP S.BceS28I BceSVIP

BsuNI BsuNI BsuNI BsuN11I
M.BsuNI BsuNIP S.BsuNI BsuN11IP

BsuN11I BsuN11I BsuN11I BsuN22I
M.BsuN11I BsuN11IP S.BsuN11I BsuN22IP

BsuN22I BsuN22I BsuN22I BsuN31I
M.BsuN22I BsuN22IP S.BsuN22I BsuN31IP

BsuN31I BsuN31I BsuN31I BsuN42I
M.BsuN31I BsuN31IP S.BsuN31I BsuN42IP

BsuN42I BsuN42I BsuN42I BsuNI
M.BsuN42I BsuN42IP S.BsuN42I BsuNIP

CjeFII CjeFII CjeFII CjeF38011II
M.CjeFII CjeFII S.CjeFII CjeF38011IIP

CjeFIV CjeFIV CjeFIV CjeFII
M.CjeFIV CjeFIV S.CjeFIV CjeFIIP

CjeF38011II CjeF38011II CjeF38011II CjeFIV
M.CjeF38011II CjeF38011IIP

'\nClearly the issue is:\n1. names have things pinned onto them. This has been managed.\n2. the target data is out of order. \n- Instead of zipping them, pick a protien name and try to find it in m,r,s,t\n- If found then add each name variation into new dictionary. \nprotien_names_use = {1:["M.Aba78I","S.Aba78I","Aba78IP","Aba78IP"], 2:[...]}\n- Now I have a can loop through the dictionary indexs 1,2,3... to pull out the names in each respective files/csv \nand pull in the data to do stuff with. This keeps it ordered and it only takes protiens present in the target.\n'

In [8]:
# create method to find protien name in other dict elements. Can I find Aba78I in m,r,s,t ? This ignore sorting.
name = "Aba78I"

protien_names_ordered_filtered = {}
for i, name_from_m in enumerate(protien_names['m']):
    name = name_from_m[2:]
    if ("M." + name) in protien_names['m']:
        if (name + "P") in protien_names['r']:
            if ("S." + name) in protien_names['s']:
                if (name + "P") in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + "P" + ".pt", "S." + name + ".pt", name + "P"] # M. / P / S. / P
                elif  name in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + "P" + ".pt", "S." + name + ".pt", name] # M. / P / S. / _
            elif ("S1." + name) in protien_names['s']:
                if (name + "P") in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + "P" + ".pt", "S1." + name + ".pt", name + "P"] # M. / P / S1. / P
                elif  name in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + "P" + ".pt", "S1." + name + ".pt", name] # M. / P / S1. / _
        elif name in protien_names['r']:
            if ("S." + name) in protien_names['s']:
                if (name + "P") in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + ".pt", "S." + name + ".pt", name + "P"] # M. / _ / S. / P
                elif  name in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + ".pt", "S." + name + ".pt", name] # M. / _ / S. / _
            elif ("S1." + name) in protien_names['s']:
                if (name + "P") in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + ".pt", "S1." + name + ".pt", name + "P"] # M. / _ / S1. / P
                elif  name in protien_names['target']:
                    protien_names_ordered_filtered[i] = ["M." + name + ".pt", name + ".pt", "S1." + name + ".pt", name] # M. / _ / S1. / _

print(protien_names_ordered_filtered.keys())


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [51]:
INPUT_DIRS_1 = [ "temp_data/m1split/", "temp_data/r1split/", "temp_data/s1split/"]

INPUT_DIRS_2 = [ "temp_data/m2split/", "temp_data/r2split/", "temp_data/s2split/" ]

def load_tensor(directory):
    tensor_dict = torch.load(directory)
    tensor_2d = tensor_dict['representations'][33]
    tensor_flat = torch.flatten(tensor_2d)
    return tensor_flat


list_combined_1d_tensors = []
for i in range(600):
    hold_flat_tensor_single_protien = [] # will store the 1d tensor from m (m1 and maybe m2), then r, then s. 
    for mrst_index, (dir1, dir2) in enumerate(zip(INPUT_DIRS_1, INPUT_DIRS_2)):
        tensor_1 = load_tensor(dir1 + protien_names_ordered_filtered[i][mrst_index])
        if os.path.isfile(dir2 + protien_names_ordered_filtered[i][mrst_index]): # if this seq was too large it will have a tensor (probably smaller) in the second file
            tensor_2 = load_tensor(dir2 + protien_names_ordered_filtered[i][mrst_index])
            combined_tensor = torch.cat( (tensor_1, tensor_2) )
        else:
            combined_tensor = tensor_1 # If there is no tensor in the second file then we only need to load the first tensor
        hold_flat_tensor_single_protien.append(combined_tensor)
    list_combined_1d_tensors.append( torch.cat( ( hold_flat_tensor_single_protien[0], hold_flat_tensor_single_protien[1], hold_flat_tensor_single_protien[2] ) ) )



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [53]:
#### get padding size
len_list = []
for list in list_combined_1d_tensors:
    len_list.append(len(list))
max_seq_len = max(len_list)



max_len_val 3462400


In [65]:
#### PAD

list_combined_1d_tensors_padded = []
for i, line in enumerate(list_combined_1d_tensors):
    #list_combined_1d_tensors_padded.append( F.pad(list_combined_1d_tensors[i], ( max_len_val-len(line), 0), "constant" ) )
    list_combined_1d_tensors_padded.append( F.pad(list_combined_1d_tensors[i], ( 0, max_len_val-len(line)), "constant" ) )



In [67]:
print(list_combined_1d_tensors_padded[0].size())
print(list_combined_1d_tensors[0].size())
print(list_combined_1d_tensors_padded[0])
print(list_combined_1d_tensors[0])

torch.Size([3462400])
torch.Size([2630400])
tensor([0.1734, 0.1142, 0.0800,  ..., 0.0000, 0.0000, 0.0000])
tensor([ 0.1734,  0.1142,  0.0800,  ...,  0.2647, -0.2722,  0.2444])


In [68]:
#### build list of tensors into single torch tensor
print("torch stacking the list of tensors into single tensor")
input_data = torch.stack(list_combined_1d_tensors_padded)

print("data.size()", input_data.size())

torch stacking the list of tensors into single tensor
data.size() torch.Size([600, 3462400])


In [ ]:
print('saving')
#torch.save(data, "data/msr-esmb1-flat.pt")
if BUILD_PARTIAL: torch.save(data[0:2], "data/PARTIALDATAmsr-esmb1-flat.pt")